#Local actions

##Preparation

In [ ]:
!mkdir TensorFlow1
!virtualenv TensorFlow1
!cd TensorFlow1
!source bin/activate
!git clone 'https://github.com/tensorflow/models.git'
!pip install tensorflow
!pip install tf_slim
!pip install pycocotools
!pip install pandas
!pip install tf-models-official
!pip install opencv-python
!pip install lvis
!pip install tensorflow_io
#!pip install tensorflow tf_slim pycocotools pandas tf-models-official opencv-python lvis tensorflow_io
#!pip install tensorflow-object-detection-api

In [ ]:
!mkdir workspace
!mkdir workspace/scripts
!mkdir workspace/scripts/preprocessing
!mkdir workspace/training_demo
!cd workspace/training_demo/
!mkdir annotations
!mkdir exported_models
!mkdir images
!mkdir images/train
!mkdir images/test
!mkdir models
!mkdir pre-trained-models

In [ ]:
!cd Tensorflow1/models/research/
protoc object_detection/protos/*.proto --python_out=.

!cp object_detection/packages/tf2/setup.py .
!pip install --use-feature=2020-resolver .

#Test
python object_detection/builders/model_builder_tf2_test.py

###Dowload or create dataset

>Download from [Open Image](https://www.v7labs.com/open-datasets/open-image-v6) and [Kaggle](https://www.kaggle.com/datasets/pavelbiz/eyes-rtte) the eyes and pupils datasets.

>Put the train and the test images into <code>Tensorflow1/workspace/training_demo/images/train</code> and <code>Tensorflow1/workspace/training_demo/images/test</code>.

>Make sure that train and test folder contain <code>*.xml</code> files for each image

> We used [labelImage](https://github.com/tzutalin/labelImg): a graphical image annotation tool and label object bounding boxes in images, very useful because it generated for each images the equivalent information (bounding boxes points xmin, ymin, xmax, ymax and the name of objects) in <code>*.xml</code>.

###Create label file

>Create the file <code>Tensorflow1/workspace/training_demo/annotations/label_map.pbtxt</code> as following:

In [ ]:
item {
    id: 1
    name: 'Human eye'
}

> Same <code>Tensorflow1/workspace/training_demo/annotations/label_map.pbtxt</code> for the pupil detection:

In [ ]:
item {
    id: 1
    name: 'pupil'
}

###Convert XML files into TFRecord

In [ ]:
!cp -r 'Tensorflow1/models/research/object_detection' 'Tensorflow1/workspace/scripts/preprocessing'

>In directory <code>Tensorflow1/workspace/scripts/preprocessing</code> execute (fill the variables with the correct paths):

>Download the utility script to <code>Tensorflow1/workspace/scripts/preprocessing</code>

In [ ]:
!curl 'https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py' --output generate_tfrecord.py

>In the same directory execute the downloaded script

In [ ]:
!python generate_tfrecord.py -x $PATH_TO_IMAGES_FOLDER/train -l $PATH_TO_ANNOTATIONS_FOLDER/label_map.pbtxt -o $PATH_TO_ANNOTATIONS_FOLDER/train.record
!python generate_tfrecord.py -x $PATH_TO_IMAGES_FOLDER/test -l $PATH_TO_ANNOTATIONS_FOLDER/label_map.pbtxt -o $PATH_TO_ANNOTATIONS_FOLDER/test.record

##Configure pretrained model

>Download the model from [TensorFlow 2 model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)

>Extract the downloaded model into <code>Tensorflow1/workspace/training_demo/pre-trained-models</code>

>Create the folder according to the downloaded model

In [ ]:
!mkdir Tensorflow1/workspace/training_demo/models/ssd_mobilenet_v2_fpn_keras

>Configure correctly the <code>pipeline.config</code> file:
* num_classes = x, where x is the total number of item for object detection;
* The batch size is the amount of samples you feed in your network;
* path.


In [ ]:
```
model {
  ssd {
    num_classes: 1 #TO CHANGE <---------------------------------
    image_resizer {
      fixed_shape_resizer {
        height: 640
        width: 640
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          random_normal_initializer {
            mean: 0.0
            stddev: 0.009999999776482582
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.996999979019165
          scale: true
          epsilon: 0.0010000000474974513
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    box_predictor {
      weight_shared_convolutional_box_predictor {
        conv_hyperparams {
          regularizer {
            l2_regularizer {
              weight: 3.9999998989515007e-05
            }
          }
          initializer {
            random_normal_initializer {
              mean: 0.0
              stddev: 0.009999999776482582
            }
          }
          activation: RELU_6
          batch_norm {
            decay: 0.996999979019165
            scale: true
            epsilon: 0.0010000000474974513
          }
        }
        depth: 128
        num_layers_before_predictor: 4
        kernel_size: 3
        class_prediction_bias_init: -4.599999904632568
        share_prediction_tower: true
        use_depthwise: true
      }
    }
    anchor_generator {
      multiscale_anchor_generator {
        min_level: 3
        max_level: 7
        anchor_scale: 4.0
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        scales_per_octave: 2
      }
    }
    post_processing {
      batch_non_max_suppression {
        score_threshold: 9.99999993922529e-09
        iou_threshold: 0.6000000238418579
        max_detections_per_class: 100
        max_total_detections: 100
        use_static_shapes: false
      }
      score_converter: SIGMOID
    }
    normalize_loss_by_num_matches: true
    loss {
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      classification_loss {
        weighted_sigmoid_focal {
          gamma: 2.0
          alpha: 0.25
        }
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    encode_background_as_zeros: true
    normalize_loc_loss_by_codesize: true
    inplace_batchnorm_update: true
    freeze_batchnorm: false
  }
}
train_config {
  batch_size: 4 #TO CHANGE <------------------------------------------
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    random_crop_image {
      min_object_covered: 0.0
      min_aspect_ratio: 0.75
      max_aspect_ratio: 3.0
      min_area: 0.75
      max_area: 1.0
      overlap_thresh: 0.0
    }
  }
  sync_replicas: true
  optimizer {
    momentum_optimizer {
      learning_rate {
        cosine_decay_learning_rate {
          learning_rate_base: 0.07999999821186066
          total_steps: 50000
          warmup_learning_rate: 0.026666000485420227
          warmup_steps: 1000
        }
      }
      momentum_optimizer_value: 0.8999999761581421
    }
    use_moving_average: false
  }
  fine_tune_checkpoint: "pre-trained-models/ssd_mobilenet_v2_fpn_keras/checkpoint/ckpt-0"  #TO CHANGE  <------------------------------"
  num_steps: 50000
  startup_delay_steps: 0.0
  replicas_to_aggregate: 8
  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
  fine_tune_checkpoint_type: "detection"  #TO CHANGE  <-------------------
  use_bfloat16: false  #TO CHANGE false = no tpu / true = si tpu  <----------
  fine_tune_checkpoint_version: V2
}
train_input_reader: {
  label_map_path: "annotations/label_map.pbtxt"  #TO CHANGE  <----------------
  tf_record_input_reader {
    input_path: "annotations/train.record"  #TO CHANGE  <-----------------
  }
}
eval_config {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
}
eval_input_reader: {
  label_map_path: "annotations/label_map.pbtxt"  #TO CHANGE  <-----------------
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "annotations/test.record"  #TO CHANGE  <-----------------
  }
}
```



##Train the model

>First copy the script

In [ ]:
!cp Tensorflow1/models/research/object_detection/model_main_tf2.py Tensorflow1/workspace/training_demo/

>Train the model

In [ ]:
!python model_main_tf2.py --model_dir=models/ssd_mobilenet_v2_fpn_keras/ --pipeline_config_path=models/ssd_mobilenet_v2_fpn_keras/pipeline.config

## Tensorboard


>We can graphically tracking the different values of our neural network with a Tensorflow tools named Tensorboard, in path: <code>/workspace/training_demo</code> execute the following command:

In [ ]:
!tensorboard --logdir=models/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/

![learning_rate](LatexPdf/img/tensorboard/learning_rate_both.png)
![loss_localization](LatexPdf/img/tensorboard/loss_localization_both.png)

# Testing the model

>We can graphically observate our results on an image directly using shell. We receive in ouput the input img we choose with the object detections boxed. Matplotlib is used.

In [ ]:
!pip install python-tk

>Python <code>test.py</code> script to use TkAgg's GUI Framework. 

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

import cv2
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl

In [ ]:

mpl.use('TkAgg')

CUSTOM_MODEL_NAME = 'training'
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpn_keras'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

paths = {
    'WORKSPACE_PATH': os.path.join('workspace','training_demo'),
    'SCRIPTS_PATH': os.path.join('scripts'),
    'APIMODEL_PATH': os.path.join('models'),
    'ANNOTATION_PATH': os.path.join('workspace','training_demo','annotations'),
    'IMAGE_PATH': os.path.join('.'),
    'MODEL_PATH': os.path.join('workspace','training_demo','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('workspace','training_demo','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('workspace','training_demo','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('workspace','training_demo','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('workspace','training_demo','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('workspace','training_demo','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('protoc')
 }

files = {
    'PIPELINE_CONFIG':os.path.join('workspace','training_demo','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME),
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'photo.jpg')

img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
print(num_detections)
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections


# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

for x in detections['detection_boxes']:
  print(x)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=20,
            min_score_thresh=.4,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()


In [ ]:
!python test.py

![detection](LatexPdf/img/latex/merge.jpg)

##Export model

>Copy <code>cp models/research/object_detection/exporter_main_v2.py workspace/training_demo/</code>

>Export model

In [ ]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path models/ssd_mobilenet_v2_fpn_keras/pipeline.config --trained_checkpoint_dir models/ssd_mobilenet_v2_fpn_keras/ --output_directory exported_models/EyeTracking_Check_2022_03_05_19-13

### Export and Convert into .tflite and quantization

> Copy from <code>Tensorflow1/models/research/objectdetection/export_tflite_graph_tf2.py</code> to <code>workspace/training_demo</code>

In [ ]:
!python export_tflite_graph_tf2.py --pipeline_config_path=ssd_mobilenet_v2_fpn_keras/pipeline.config --trained_checkpoint_dir=ssd_mobilenet_v2_fpn_keras/ --output_directory=ssd_mobilenet_v2_fpn_keras/tflite

>Python <code>converter.py</code> script for converting and quantization

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [ ]:
saved_model_dir = 'pathtosaved_model/saved_model'

# Representative_dataset function
def representative_dataset_int():
    for _ in range(100):
      data = np.random.rand(1, 244, 244, 3)
      yield [data.astype(np.uint)]

def representative_dataset_f32():
    for _ in range(100):
      data = np.random.rand(1, 244, 244, 3)
      yield [data.astype(np.float32)]

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir) # path to the SavedModel directory
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# BELOW JUST IN CASE FOR CONVERTING FROM FLOAT32 TO UINT8
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8,tf.lite.OpsSet.TFLITE_BUILTINS]
# converter.representative_dataset = representative_dataset_f32

tflite_model = converter.convert()


>Saving the model

In [ ]:
# Save the model.
with open('converted_model_DFL.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
!python converter.py

# Metadata File


>Tflite requires a metadata file for a correct functioning during Android's operations when Input tensor has type kTfLiteFloat32: it requires specifying NormalizationOptions metadata to preprocess input images. 
This file is not required when input tensor has type uint8. [Tflite official guide](https://www.tensorflow.org/lite/convert/metadata?hl=en)

>We generate a new <code>Tensorflow1/workspace/training_demo/annotations/labelmap.txt</code> file containing for each row our items. In our case, we have only one row for our 'Human eye'. 

In [ ]:
from tflite_support.metadata_writers import object_detector
from tflite_support.metadata_writers import writer_utils
from tflite_support import metadata

ObjectDetectorWriter = object_detector.MetadataWriter
_MODEL_PATH = "eyemodel.tflite"
_LABEL_FILE = "labelmap.txt"
_SAVE_TO_PATH = "eyemodel_metadata.tflite"

writer = ObjectDetectorWriter.create_for_inference(
    writer_utils.load_file(_MODEL_PATH), [127.5], [127.5], [_LABEL_FILE])
writer_utils.save_file(writer.populate(), _SAVE_TO_PATH)

# Verify the populated metadata and associated files.
displayer = metadata.MetadataDisplayer.with_model_file(_SAVE_TO_PATH)
print("Metadata populated:")
print(displayer.get_metadata_json())
print("Associated file(s) populated:")
print(displayer.get_packed_associated_file_list())

>Now, we import this new <code>eyemodel_metadata.tflite</code> to our Android App as a Machine Learning [ml](https://github.com/DaniDF/sistemiDigitali2022/tree/master/Android/app/src/main/ml) file.